In [1]:
from fastapi import FastAPI
from pydantic import BaseModel
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from openai import OpenAI
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_json_chat_agent
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


In [2]:
# chatbot 관련 변수 설정
embeddings = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)

vectorstore =  Chroma(persist_directory="./testdb", embedding_function=embeddings)

retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={"k": 5})   

retriever_tool = create_retriever_tool(
    retriever,
    name="db_search",
    description="놀이 관련 정보를 연결된 데이터베이스에서 검색합니다.",
)
tools = [retriever_tool]

llama3 = ChatOpenAI(
    base_url="http://sionic.chat:8001/v1",
    api_key="934c4bbc-c384-4bea-af82-1450d7f8128d",
    model="xionic-ko-llama-3-70b",
    temperature=0.1,
)

json_prompt = hub.pull("teddynote/react-chat-json-korean")

llama3_agent = create_json_chat_agent(llama3, tools, json_prompt)

llama3_agent_executor = AgentExecutor(
    agent=llama3_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True,
)



c:\Users\User\anaconda3\envs\llama\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\User\anaconda3\envs\llama\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
response = llama3_agent_executor.invoke(
        {'input': '0세 아기와 할 수 있는 놀이?'},
        #  config={'configurable': {'session_id': session_id}},
    )



> Entering new AgentExecutor chain...
```
{
    "action": "db_search",
    "action_input": "0세 아기와 할 수 있는 놀이"
}
``````
{
    "action": "db_search",
    "action_input": "0세 아기와 할 수 있는 놀이"
}
``````
{
    "action": "db_search",
    "action_input": "0세 아기와 할 수 있는 놀이"
}
```

KeyboardInterrupt: 

: 